머신 러닝 모델은 여러 가지 방법으로 예측 성능을 평가할 수 있습니다. 모델의 목적과 데이터의 종류에 따라 맞는 평가 방법을 사용하지 않는다면 잘못된 평가 결과에 빠질 수 있습니다. 이번 장에서는 분류 모델에 사용되는 성능 평가 지표인 정확도(Accuracy), 오차행렬(Confusion Matrix), 정밀도(Precision), 재현율(Recall), F1 스코어, ROC AUC 에 대해 설명합니다.

## 1. 정확도 (Accuracy)

정확도는 실제 데이터에서 예측 데이터가 얼마나 같은지를 판단하는 지표입니다.
$$ 정확도(Accuracy) = \frac{예측\;결과가\;동일한\;데이터\;건수}{ 전체\;예측\;데이터\;건수 } $$

정확도는 직관적으로 모델 예측 성능을 나타내는 평가 지표입니다. 하지만 불균형한 레이블 값 분포에서 ML 모델의 성능을 판단할 경우 적합한 평가 지표가 아닙니다. 예를 들어 100 개의 데이터가 있고 이중 90개의 데이터 레이블이 0, 단 10 개의 레이블이 1이라 한다면 무조건 0으로 예측 결과를 반환하는 ML모델의 경우라도 정확도가 90%가 됩니다.

0-9까지의 숫자 이미지의 픽셀 정보 데이터 세트인 `MNIST` 데이터 세트를 이용해 실습을 해보겠습니다. `MNIST` 데이터 세트 중 레이블 값이 7 인 것만 True, 나머지를 False 로 변환해 전체 데이터의 10%만 True, 90는 False 인 불균형한 데이터 세트로 변환해 본 뒤, 모든 데이터에 대해 0을 결과값으로 반환하는 `MyFakeClassifier`를 이용해 에측과 평가를 수행해 보겠습니다.

In [2]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X ,y):
        pass
    
    #입력 값으로 들어오는 X 데이터 세트의 크기 만큼 모두 0 값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X),1), dtype=bool)

# MNIST 데이터 세트를 불러와서 타깃 레이블이 7 인 경우 1, 아니라면 0으로 변환
digits= load_digits()
y= (digits.target==7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# 헉숩 / 예측 / 정확도 평가
fakeclf= MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred=fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test,fakepred)))

레이블 테스트 세트 크기 : (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는:0.900


모든 값을 0 으로 예측하여도 90%의 정확도가 나오는 것을 확인할 수 있습니다. 이렇게 정확도 평가 지표를 불균형한 레이블 데이터 세트에서 사용했을 때의 문제점을 해결하기 위해 여러 분류 지표를 함께 사용하여야 합니다,

## 2. 오차 행렬

이진 분류에서 성능 지표로 활용되는 오차 행렬 (confusion matrix, 혼동 행렬)은 학습된 분류 모델의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지를 함께 나타내는 지표입니다. 오차 행렬은 4분면 행렬에서 실제 레이블 클래스 값과 예측 레이블 클래스 값이 어떠한 유형을 가지고 매핑되는지를 나타냅니다. 4분면의 왼쪽, 오른쪽을 예측된 클래스 값 기준으로 Negative 와 Positive 로 분류하고, 4분면의 위, 아래를실제 클래스 값 기준으로 Negative 와 Positive로 분류하면 예측 클래스와 실제 클래스의 값 유형에 따라 결정되는 TN, FP, FN, TP 형태로 오차 행렬의 4분면을 채울 수 있습니다. 

> TN는 예측 값을 Negative 값 0 으로 에측했고 실제 값 역시 Negative 값 0

> FP는 예측 값을 Positive 값 1 으로 에측했고 실제 값 역시 Negative 값 0

> FN는 예측 값을 Negative 값 0 으로 에측했고 실제 값 역시 Positive 값 1

> TP는 예측 값을 Positive 값 1 으로 에측했고 실제 값 역시 Positive 값 1

사이킷런은 오차 행렬을 구하기 위해 confusion_matrix() API를 제공합니다. 앞서 정확도 에서 사용한 예제를 그대로 활용하여 오차 행렬을 출력해 보겠습니다.

In [3]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]])

출력 결과 TN, FP, FN, TP의 개수를 각각 확인할 수 있습니다. 이 값들을 조합하면 여러 성능 측정 지표인 정확도(Accuracy), 정밀도(Precision), 재현율(Recall) 값을 알 수 있습니다.
예를 들어, 정확도는 다음과 같이 계산이 가능합니다.

$$ 정확도(Accuracy) = \frac{예측\;결과가\;동일한\;데이터\;건수}{ 전체\;예측\;데이터\;건수 } = \frac{TN + TP}{ TN + FP + FN + TP }$$ 

## 3. 정밀도와 재현율

정밀도와 재현율은 Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가 지표입니다. 각각 다음과 같은 공식으로 계산됩니다.

$$ 정밀도(Precision) = \frac{예측과\;실제\;값이\;모두\;Positive인\;데이터\;건수}{ 예측을\; Positive로\;한\;데이터\;건수 } = \frac{TP}{ FP + TP }$$ 
$$ 재현율(Recall) = \frac{예측과\;실제\;값이\;모두\;Positive인\;데이터\;건수}{ 실제\; 값이\; Positive인\;데이터\;건수 } = \frac{TP}{ FN + TP }$$ 

정밀도와 재련율 지표는 분류 모델의 목적에 따라 특정 평가 지표가 더 중요한 지표로 간주될 수 있습니다. 실제 Positive인 데이터 예측을 Negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우에는 재현율이 상대적으로 더 중요한 지표입니다. 한편 실제 Negative인 데이터 예측을 Positive로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우에는 정밀도가 상대적으로 더 중요한 지표입니다. 재현율과 정밀도 모두 TP를 높이는 데 동일하게 초점을 맞추지만, 재현율은 FN을 낮추는 데, 정밀도는 FP를 낮추는 데 초점을 맞추기 때문입니다.

일반적인 경우 두 지표는 서로 보완적인 지표로 분류의 성능을 평가하는 데 적용되며, 어느 한쪽만 높은 수치를 얻기보다는 둘 모두 높은 수치를 얻는 것이 좋은 평가입니다. 

사이킷 런의 `precision_score()` 를 사용하여 정밀도를 ,`recall_score()`를 사용하여 재현율을 계산할 수 있습니다. 타이타닉 예제에 오차 행렬 및 정밀도, 재현율을 모두 구해 예측 성능을 평가해 보겠습니다.

[데이터셋 출처](https://www.kaggle.com/c/titanic/data)

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

#오차 행렬, 정확도, 정밀도, 재현율을 모두 구하는 함수
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall))

In [5]:
from sklearn import preprocessing

#앞 단원에서 사용한 데이터 전처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

def format_features(df):
    df['Cabin']=df['Cabin'].str[:1]
    features=['Cabin' , 'Sex' , 'Embarked']
    for feature in features:
        le=preprocessing.LabelEncoder()
        le=le.fit(df[feature])
        df[feature]=le.transform(df[feature])
    return df

def transform_features(df):
    df=fillna(df)
    df=drop_features(df)
    df=format_features(df)
    return df

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#원본 데이터 재로딩, 가공, 학습/테스트 데이터 분할
titanic_df = pd.read_csv('./train.csv')
y_titanic_df=titanic_df['Survived']
X_titanic_df=titanic_df.drop('Survived', axis=1)
X_titanic_df=transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,test_size=0.20, random_state=11)
lr_clf=LogisticRegression()

#학습 및 예측
lr_clf.fit(X_train,y_train)
pred=lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

오차 행렬
[[104  14]
 [ 13  48]]
정확도: 0.849162, 정밀도: 0.7742, 재현율: 0.7869


/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


오차 행렬과 정확도, 정밀도, 재현율 값을 확인할 수 있습니다. 재현율이 정밀도보다 높게 나왔습니다. 만약 정밀도를 좀 더 강화하고 싶다면, 어떻게 해야 할까요?

### 3.1. 정밀도 - 재현율 트레이드오프

In [ ]:
import numpy as np
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)

print('pred_proba() 결과 Shape : {0}'.format(pred_proba.shape))
print('pred_proba array에서 앞 3개만 샘플로 추출 \n:',pred_proba[:3])

pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1,1)], axis=1)
print('두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n', pred_proba_result[:3])  

In [ ]:
from sklearn.metrics import precision_recall_curve

pred_proba_class1=lr_clf.predict_proba(X_test)[:, 1]

precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_class1)
print('반환된 분류 결정 임계값 배열의 Shape:', thresholds.shape)

thr_index= np.arange(0, thresholds.shape[0], 15)
print('샘플 추출을 위한 임계값 배열의 index 10개:', thr_index)
print('샘플용 10개의 임곗값: ', np.round(thresholds[thr_index],2))

print('샘플 임계값별 정밀도: ', np.round(precisions[thr_index], 3))
print('샘플 임계값별 재현율: ', np.round(recalls[thr_index], 3))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

def precision_recall_curve_plot(y_test, pred_proba_c1):
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)
    
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')
    
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    
    plt.xlabel('Threshold value')
    plt.ylabel('Precision and Recall value')
    plt.show()
    
precision_recall_curve_plot(y_test, lr_clf.predict_proba(X_test)[:, 1])

## F1 Score

In [ ]:
from sklearn.preprocessing import Binarizer
from sklearn.metrics import f1_score

def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임계값:', custom_threshold)
        get_clf_eval(y_test, custom_predict)
    


def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test,pred)
    
    f1= f1_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}\n'.format(accuracy, precision, recall, f1))
    
thresholds=[0.4, 0.45, 0.5, 0.55, 0.6]
pred_proba= lr_clf.predict_proba(X_test)
get_eval_by_threshold(y_test, pred_proba[:, 1].reshape(-1, 1), thresholds)

## ROC Curve & AUC Score

In [ ]:
from sklearn.metrics import roc_curve

def roc_curve_plot(y_test, pred_proba_c1):

    fprs, tprs, thresholds = roc_curve(y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    
    plt.plot([0,1],[0,1], 'k--', label='random')
    
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)')
    plt.ylabel('TPR(Recall)')
    plt.legend()
    
roc_curve_plot(y_test, pred_proba[:,1])

In [ ]:
from sklearn.metrics import roc_auc_score

pred_proba = lr_clf.predict_proba(X_test)[:,1]
roc_score=roc_auc_score(y_test, pred_proba)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

## 피마 인디언 당뇨병 예측

[데이터셋 출처](https://www.kaggle.com/uciml/pima-indians-diabetes-database)

### 데이터 분석

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

diabetes_data= pd.read_csv('diabetes.csv')
print(diabetes_data['Outcome'].value_counts())
diabetes_data.head(3)

In [ ]:
diabetes_data.info()

In [ ]:
diabetes_data.describe()

## 데이터 전처리

In [ ]:
zero_features=['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

mean_zero_features = diabetes_data[zero_features].mean()
diabetes_data[zero_features]=diabetes_data[zero_features].replace(0, mean_zero_features)


### 예측 모델(로지스틱 회귀)

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test,pred)
    f1= f1_score(y_test, pred)
    roc_auc= roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
    
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\ F1:{3:.4f}, AUC:{4:.4f}\n'.format(accuracy, precision, recall, f1,roc_auc))

In [ ]:
X = diabetes_data.iloc[:, :-1]
y = diabetes_data.iloc[:, -1]

scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=156, stratify=y)

lr_clf=LogisticRegression()
lr_clf.fit(X_train, y_train)
pred= lr_clf.predict(X_test)
pred_proba = lr_clf.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, pred, pred_proba)

In [ ]:
pred_proba_c1 = lr_clf.predict_proba(X_test)[:,1]
precision_recall_curve_plot(y_test, pred_proba_c1)

In [ ]:
binarizer = Binarizer(threshold=0.48)

pred_th_048 =binarizer.fit_transform(pred_proba[:, 1].reshape(-1,1))

get_clf_eval(y_test, pred_th_048, pred_proba[:, 1])